In [25]:
#-*- coding: utf-8 -*-
# 라이브러리 import

try:
    from selenium import webdriver
except:
    !pip install selenium
    from selenium import webdriver

try:
    from webdriver_manager.chrome import ChromeDriverManager
except:
    !pip install webdriver_manager
    from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys

try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
    from bs4 import BeautifulSoup

import datetime
from dateutil.relativedelta import relativedelta
try:
    from urllib import parse
except:
    !pip install urllib3
    from urllib import parse

import time

try:
    import csv
except:
    !pip install csv
    import csv

try:
    import getpass
except:
    !pip install getpass4
    import getpass

import re

In [26]:
# 브라우저 옵션
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('start-maximized')
options.add_argument(
            "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        )
options.add_argument("lang=ko_KR")

In [27]:
# 비밀번호 마스킹 처리 함수
def printPass(password) :
    masking_password = ""
    for s in password :
        masking_password += "·"
    return masking_password

In [28]:
# 로그인 처리 함수 loginType F = 페이스북 계정 로그인, I = 인스타 계정 로그인
def instaLogin(chrome, loginType="F", email = None, password = None) :
    chrome.get("https://www.instagram.com/accounts/login/")
    time.sleep(3)
    insta_formObj = chrome.find_element_by_css_selector("form[id=loginForm]")
    emailObj = None
    passObj = None
    loginBtnObj = None
    if loginType == "I":
        print("loginType = I")
        emailObj = insta_formObj.find_element_by_css_selector("input[name=username]")
        passObj = insta_formObj.find_element_by_css_selector("input[name=password]")
        loginBtnObj = insta_formObj.find_element_by_css_selector("button[type=submit]")
    else :
        print("loginType = F")
        loginMoveBtnObj = insta_formObj.find_element_by_css_selector("button.sqdOP.yWX7d.y3zKF[type=button]")
        loginMoveBtnObj.click()
        time.sleep(3)
        facebook_formObj = chrome.find_element_by_css_selector("form[id=login_form]")
        emailObj = facebook_formObj.find_element_by_css_selector("input[id=email]")
        passObj = facebook_formObj.find_element_by_css_selector("input[id=pass]")
        loginBtnObj = facebook_formObj.find_element_by_css_selector("button[id=loginbutton]")

    if email is None :
        email = input("이메일 또는 아이디 : ")
    else :
        print("이메일 또는 아이디 : " + email)

    if password is None :
        password = getpass.getpass("비밀번호 : ")
    else :
        print("비밀번호 : " + printPass(password))

    emailObj.clear()
    emailObj.send_keys(email)
    passObj.clear()
    passObj.send_keys(password)
    time.sleep(1)
    loginBtnObj.click()
    time.sleep(5)


In [29]:
# 페이스북 로그인 처리 체크 후 재처리를 위한 함수
def loginFacebookCheck(chrome) :
    if chrome.current_url == "https://www.instagram.com/accounts/login/":
        insta_formObj = chrome.find_element_by_css_selector("form[id=loginForm]")
        loginMoveBtnObj = insta_formObj.find_element_by_css_selector("button.sqdOP.yWX7d.y3zKF[type=button]")
        loginMoveBtnObj.click()
        time.sleep(5)
        return True
    else :
        return False


In [30]:
# 첫번째 사진 선택 함수
def selectFirst(chrome):
    try:
        first = chrome.find_element_by_css_selector("div._9AhH0")
        first.click()
        time.sleep(2)
        return True
    except Exception as e:
        print(e)
        return False


In [31]:
# 다음 사진으로 이동 함수
def moveNext(chrome):
    try:
        right = chrome.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
        right.click()
        time.sleep(3)
        return True
    except Exception as e:
        print(e)
        return False


In [32]:
# 내용 추출 함수
def getContent(chrome):
    
    html = chrome.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자 
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > a > span')[0].text   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    return data

In [33]:
# 크롤링 시작 함수
def startSearch(loginType = "F", email = None, password = None) :
    search_txt = input("search : ")
    encode_type = "utf-8"
    search_encode = parse.quote(search_txt)
    chrome = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    try:
        instaLogin(chrome, loginType, email, password)
        time.sleep(2)
        
        #print(chrome.current_url)
        searchUrl = "https://www.instagram.com/explore/tags/" + search_encode + "/"
        
        chrome.get(searchUrl)
        time.sleep(3)
        
        # Facebook 로드인 연동이 안되는 경우가 있어 Facebook 재 연동처리
        if loginType == "F" :
            if loginFacebookCheck(chrome) :
                chrome.get(searchUrl)
                time.sleep(3)
                
        
        now_date = datetime.datetime.now()

        cvsFileName = "inta_search_" + search_txt + "_" + now_date.strftime("%Y-%m-%d") + ".csv"
        with open(cvsFileName, "w", newline="", encoding=encode_type) as f:
            colnames = ['content', 'date', 'like', 'place', 'tags']
            w = csv.writer(f)
            w.writerow(colnames)
        count = 0
        if selectFirst(chrome) :
            while True :
                count = count+1
                with open(cvsFileName, "a", newline="", encoding=encode_type) as f:
                    w = csv.writer(f)
                    data = getContent(chrome)
                    print("[" + str(count) + "] : ", data)
                    w.writerow(data)
                if moveNext(chrome) :
                    time.sleep(2)
                else :
                    break;

    except Exception as e:
        print(e)
    finally:
        if chrome:
            chrome.close()

In [34]:
startSearch(email = "gbible@naver.com")

search :  복권당첨


[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [C:\Users\gbible\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


 
loginType = F
이메일 또는 아이디 : gbible@naver.com


비밀번호 :  ·········


[1] :  ['#4등#로또#로또당첨5등 말고 처음 되봄하나만 더 맞지..#곱창 사 먹어야징~', '2018-09-17', '178', '', ['#4등', '#로또', '#로또당첨5등', '#곱창']]
[2] :  ['-우연히 천원짜리 로또 딱 한장샀는데3등이 당첨됐,,,,,,,,, #대박 🎉🎊우리도 이런운이있네? 싱기방기😜#로또 #로또당첨#천원의행복 #yolo', '2017-06-13', '123', '', ['#대박', '#로또', '#로또당첨', '#천원의행복', '#yolo']]
[3] :  ['운수 좋은 날 💸💸✨', '2019-09-14', '124', 'Hwacheon', []]
[4] :  ['집사님덜 제가 오늘 끄니 수염을 주웠거든요? 꺄 십만원 당첨!!! 끄니가 사료값 벌어왔어요\U0001f929\U0001f929', '2019-03-09', '133', '', []]
[5] :  ['저번주 꿈에뭔 물청소를 하길래꿈풀이 해보니 상당히 좋아서복권 엄청샀는데 역시 나는횡재운은 없는듯?ㅋㅋ열심히 일하자🙆\u200d♂️.#복권 #로또 #복권당첨 #꿈 #꿈해몽 #연금복권 #꽝 #전주맛집 #맛집추천 #존맛탱 #셀카그램 #셀카스타그램 #셀스타그램 #셀피 #먹방스타그램 #먹스타그램 #선팔환영 #선팔맞팔 #선팔하면맞팔 #럽스타그램 #월요일 #맛스타 #맛스타그램 #전주맛집추천', '2019-12-23', '106', 'Jeonju', ['#복권', '#로또', '#복권당첨', '#꿈', '#꿈해몽', '#연금복권', '#꽝', '#전주맛집', '#맛집추천', '#존맛탱', '#셀카그램', '#셀카스타그램', '#셀스타그램', '#셀피', '#먹방스타그램', '#먹스타그램', '#선팔환영', '#선팔맞팔', '#선팔하면맞팔', '#럽스타그램', '#월요일', '#맛스타', '#맛스타그램', '#전주맛집추천']]


WebDriverException: Message: chrome not reachable
  (Session info: chrome=90.0.4430.93)
